In [30]:
!pip install -i https://test.pypi.org/simple/ krovetz

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Looking in indexes: https://test.pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 3.6 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for krovetz: filename=krovetz-1.0.0-cp310-cp310-linux_x86_64.whl size=289877 sha256=e81e44bf4ebe913c60701e1ce65627af19c3409defa9dc429d1e0070d2372637
  Stored in directory: /root/.cache/pip/wheels/4b/8f/7b/ba27616be01a2d9b8281a280d25acc77f20ab605368f2d607d
Successfully built krovetz


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
df=pd.read_csv('/kaggle/input/myanimelist-dataset/anime-filtered.csv')
df.head()

,anime_id,Name,Score,Genres,English name,Japanese name,sypnopsis,Type,Episodes,Aired,...,Duration,Rating,Ranked,Popularity,Members,Favorites,Watching,Completed,On-Hold,Dropped
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,"In the year 2071, humanity has colonized sever...",TV,26,"Apr 3, 1998 to Apr 24, 1999",...,24 min. per ep.,R - 17+ (violence & profanity),28.0,39,1251960,61971,105808,718161,71513,26678
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,"other day, another bounty—such is the life of ...",Movie,1,"Sep 1, 2001",...,1 hr. 55 min.,R - 17+ (violence & profanity),159.0,518,273145,1174,4143,208333,1935,770
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,"Vash the Stampede is the man with a $$60,000,0...",TV,26,"Apr 1, 1998 to Sep 30, 1998",...,24 min. per ep.,PG-13 - Teens 13 or older,266.0,201,558913,12944,29113,343492,25465,13925
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),ches are individuals with special powers like ...,TV,26,"Jul 2, 2002 to Dec 24, 2002",...,25 min. per ep.,PG-13 - Teens 13 or older,2481.0,1467,94683,587,4300,46165,5121,5378
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,It is the dark century and the people are suff...,TV,52,"Sep 30, 2004 to Sep 29, 2005",...,23 min. per ep.,PG - Children,3710.0,4369,13224,18,642,7314,766,1108


In [27]:
df.dropna(inplace=True)
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13229 entries, 0 to 13228
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   anime_id       13229 non-null  int64  
 1   Name           13229 non-null  object 
 2   Score          13229 non-null  float64
 3   Genres         13229 non-null  object 
 4   English name   13229 non-null  object 
 5   Japanese name  13229 non-null  object 
 6   sypnopsis      13229 non-null  object 
 7   Type           13229 non-null  object 
 8   Episodes       13229 non-null  object 
 9   Aired          13229 non-null  object 
 10  Premiered      13229 non-null  object 
 11  Producers      13229 non-null  object 
 12  Licensors      13229 non-null  object 
 13  Studios        13229 non-null  object 
 14  Source         13229 non-null  object 
 15  Duration       13229 non-null  object 
 16  Rating         13229 non-null  object 
 17  Ranked         13229 non-null  float64
 18  Popula

In [29]:
print(df['sypnopsis'][103])

Taka (technically Tamahome) and Miaka are now happily married and only six months before their new family would be born, but an obsessed girl named Mayo would do anything to make Taka hers. She enters the world of the Four Gods, determined to win Taka's heart and be the new priestess.


In [6]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW,pipeline
import torch
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
import re
import numpy as np 
from torch.utils.data import DataLoader, Dataset

In [7]:
contraction_mapping = {"ain't": "is not", "its": "it is","aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [8]:
stop_words = set(stopwords.words('english')) 
def text_cleaner(text):
    newString = text.lower()
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    tokens = [w for w in newString.split() if not w in stop_words]
    long_words=[]
    for i in tokens:
        if len(i)>=3:                  #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()

In [ ]:
cleaned_text = []
for t in df['sypnopsis']:
    cleaned_text.append(text_cleaner(t))

In [ ]:
for i in range(5):
    print(cleaned_text[i])
    print('')

In [ ]:
from keybert import KeyBERT
from nltk.stem import PorterStemmer
# Load the KeyBERT model (based on BERT embeddings)
kw_model = KeyBERT()
title_keywords={}
for i in range(len(cleaned_text)):
    text = cleaned_text[i]
    kw=[]
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None,top_n=25)

    kw.append(keywords)
    title = df['Name'][i]
    title_keywords[title]=kw
    


In [ ]:
print(title_keywords[df['Name'][10]])

In [ ]:
print(cleaned_text[10])

In [ ]:
print(df['Name'][df['Name']=="Naruto"])

In [ ]:
keywords_df=pd.DataFrame(columns=["Title","Keywords"])

In [ ]:
for key,value in title_keywords.items():
    temp={}
    temp['Title']=key
    temp['Keywords']=value
    new_row = pd.DataFrame(temp)
    keywords_df = pd.concat([keywords_df, new_row], ignore_index=True)

In [ ]:
keywords_df.head()

In [ ]:
keywords_df.to_csv("/kaggle/working/keywords_title.csv")

In [9]:
keywords_df=pd.read_csv("/kaggle/input/keywords-title-content-filtering/keywords_title.csv")
keywords_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13228 entries, 0 to 13227
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  13228 non-null  int64 
 1   Title       13228 non-null  object
 2   Keywords    13228 non-null  object
dtypes: int64(1), object(2)
memory usage: 310.2+ KB


In [10]:
keywords_df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [11]:
keywords_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13228 entries, 0 to 13227
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Title     13228 non-null  object
 1   Keywords  13228 non-null  object
dtypes: object(2)
memory usage: 206.8+ KB


In [12]:
keywords_df.head()

,Title,Keywords
0,Cowboy Bebop,"[('moons', 0.4515), ('spaceship', 0.4177), ('p..."
1,Cowboy Bebop: Tengoku no Tobira,"[('bounty', 0.5031), ('crew', 0.3781), ('casua..."
2,Trigun,"[('vash', 0.4791), ('stampede', 0.4457), ('vil..."
3,Witch Hunter Robin,"[('witches', 0.5465), ('witch', 0.4666), ('tel..."
4,Bouken Ou Beet,"[('zenon', 0.4466), ('century', 0.3654), ('van..."


In [18]:
import ast
keywords_df['Keywords'] = keywords_df['Keywords'].apply(ast.literal_eval)

print(type(keywords_df['Keywords'][0]))

<class 'list'>


In [ ]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()

In [ ]:
for i in range(13228):
    words=keywords_df['Keywords'][i]
    for j in range(25):
        if len(words)<25:
            print(i)
            continue
        else:
            words[j]=list(words[j])
    keywords_df['Keywords'][i]=words
print(keywords_df['Keywords'][0])